# Train Word2Vec model
This code can be found in the TensorFlow tutorial code [here](https://www.tensorflow.org/tutorials/text/word2vec) <br>
There is much more informational and educational detail in the TF tutorial so make sure take a look at that. <br>
We will just use the key parts of the code here so that we can focus on what we need too product static word embeddings <br>

In [ ]:
import io
import itertools
import numpy as np
import pandas as pd
import os
import re
import string
import tensorflow as tf
import tqdm
import psutil

from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Activation, Dense, Dot, Embedding, Flatten, GlobalAveragePooling1D, Reshape
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
import plotly.express as px

In [ ]:
# Make sure you are running TF version 2 or later
# You can find installation steps here: https://www.tensorflow.org/install
print(tf.__version__)

In [ ]:
SEED = 42 
AUTOTUNE = tf.data.experimental.AUTOTUNE
# Set the number of negative samples per positive context. 
num_ns = 4
# Set you API token as an env variable or else (not recommended) just add it directly to your notebook
NEPTUNE_API_TOKEN = (os.getenv('NEPTUNE_API'))

## Define your neptune credentials 
This will enable you to track your experiment

In [ ]:
import neptune
from neptunecontrib.api import log_table
from neptunecontrib.api import log_chart

neptune.init(project_qualified_name='<username>/sandbox',
             api_token=NEPTUNE_API_TOKEN, 
            )

## Create an experiment

In [ ]:
neptune.create_experiment()

## Set the home dir
This is where you are running your notebooks

In [ ]:
HOME_DIR = os.getcwd()

### Generate training data

In [ ]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for vocab_size tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence, 
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)
    
    # Iterate over each positive skip-gram pair to produce training examples 
    # with positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1, 
          num_sampled=num_ns, 
          unique=True, 
          range_max=vocab_size, 
          seed=SEED, 
          name="negative_sampling")
      
      # Build context and label vectors (for one target word)
      negative_sampling_candidates = tf.expand_dims(
          negative_sampling_candidates, 1)

      context = tf.concat([context_class, negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

## Prepare training data for Word2Vec

### Get the data we generated for out test lanugage dataset

In [ ]:
path_to_file = f'{HOME_DIR}/proximity_vocab.txt'
# path_to_file = f'{HOME_DIR}/random_vocab.txt'

In [ ]:
with open(path_to_file) as f: 
  lines = f.read().splitlines()
for line in lines[:5]:
  print(line)

Use the non empty lines to construct a `tf.data.TextLineDataset` object for next steps.

In [ ]:
text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))

### Vectorize sentences from the corpus
The steps here set everything to one case and also remove punctuation. <br>
We do not need these for our dataset but you will need them if using other datasets with 'real' data

In [ ]:
# We create a custom standardization function to lowercase the text and 
# remove punctuation.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')

# Define the vocabulary size and number of words in a sequence.
vocab_size = 28
sequence_length = 7

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Set output_sequence_length length to pad all samples to same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

Call `adapt` on the text dataset to create vocabulary.


In [ ]:
vectorize_layer.adapt(text_ds.batch(1024))

Once the state of the layer has been adapted to represent the text corpus, the vocabulary can be accessed with `get_vocabulary()`. This function returns a list of all vocabulary tokens sorted (descending) by their frequency. 

In [ ]:
# Save the created vocabulary for reference.
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

In [ ]:
s = set([])
for l in inverse_vocab:
    s.update([x for x in l.split()])
print(f'Full list: {sorted(s)}')
print(f'Vocab size: {len(s)}')


The vectorize_layer can now be used to generate vectors for each element in the `text_ds`.

In [ ]:
def vectorize_text(text):
    text = tf.expand_dims(text, -1)
    return tf.squeeze(vectorize_layer(text))

# Vectorize the data in text_ds.
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

### Obtain sequences from the dataset

You now have a `tf.data.Dataset` of integer encoded sentences. To prepare the dataset for training a Word2Vec model, flatten the dataset into a list of sentence vector sequences. This step is required as you would iterate over each sentence in the dataset to produce positive and negative examples. 

Note: Since the `generate_training_data()` defined earlier uses non-TF python/numpy functions, you could also use a `tf.py_function` or `tf.numpy_function` with `tf.data.Dataset.map()`.

In [ ]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))

Take a look at few examples from `sequences`.


In [ ]:
for seq in sequences[:5]:
  print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

### Generate training examples from sequences

`sequences` is now a list of int encoded sentences. Just call the `generate_training_data()` function defined earlier to generate training examples for the Word2Vec model. To recap, the function iterates over each word from each sequence to collect positive and negative context words. Length of target, contexts and labels should be same, representing the total number of training examples.

In [ ]:
targets, contexts, labels = generate_training_data(
    sequences=sequences, 
    window_size=2, 
    num_ns=4, 
    vocab_size=vocab_size, 
    seed=SEED)
print(len(targets), len(contexts), len(labels))

### Configure the dataset for performance

To perform efficient batching for the potentially large number of training examples, use the `tf.data.Dataset` API. After this step, you would have a `tf.data.Dataset` object of `(target_word, context_word), (label)` elements to train your Word2Vec model!

In [ ]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

Add `cache()` and `prefetch()` to improve performance.

In [ ]:
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

## Model and Training

### Subclassed Word2Vec Model

Use the [Keras Subclassing API](https://www.tensorflow.org/guide/keras/custom_layers_and_models) to define your Word2Vec model with the following layers:


* `target_embedding`: A `tf.keras.layers.Embedding` layer which looks up the embedding of a word when it appears as a target word. The number of parameters in this layer are `(vocab_size * embedding_dim)`.
* `context_embedding`: Another `tf.keras.layers.Embedding` layer which looks up the embedding of a word when it appears as a context word. The number of parameters in this layer are the same as those in `target_embedding`, i.e. `(vocab_size * embedding_dim)`.
* `dots`: A `tf.keras.layers.Dot` layer that computes the dot product of target and context embeddings from a training pair.
* `flatten`: A `tf.keras.layers.Flatten` layer to flatten the results of `dots` layer into logits.

With the sublassed model, you can define the `call()` function that accepts `(target, context)` pairs which can then be passed into their corresponding embedding layer. Reshape the `context_embedding` to perform a dot product with `target_embedding` and return the flattened result.

Key point: The `target_embedding` and `context_embedding` layers can be shared as well. You could also use a concatenation of both embeddings as the final Word2Vec embedding.

In [ ]:
class Word2Vec(Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = Embedding(vocab_size, 
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding", )
    self.context_embedding = Embedding(vocab_size, 
                                       embedding_dim, 
                                       input_length=num_ns+1)
    self.dots = Dot(axes=(3,2))
    self.flatten = Flatten()

  def call(self, pair):
    target, context = pair
    we = self.target_embedding(target)
    ce = self.context_embedding(context)
    dots = self.dots([ce, we])
    return self.flatten(dots)

### Define loss function and compile model


For simplicity, you can use `tf.keras.losses.CategoricalCrossEntropy` as an alternative to the negative sampling loss. If you would like to write your own custom loss function, you can also do so as follows:

``` python
def custom_loss(x_logit, y_true):
      return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)
```

It's time to build your model! Instantiate your Word2Vec class with an embedding dimension of 128 (you could experiment with different values). Compile the model with the `tf.keras.optimizers.Adam` optimizer. 

In [ ]:
embedding_dim = 128
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Also define a callback to log training statistics for [Neptune](https://neptune.ai/blog/how-to-monitor-machine-learning-and-deep-learning-experiments).

In [ ]:
def get_data(weights, vocab):
    X = []
    X_vocab = []
    # Align the words and weights
    for index, word in enumerate(vocab):
        if  index == 0: continue # skip 0, it's padding.
        X.append(weights[index]) 
        X_vocab.append(word)
    return(X, X_vocab)

In [ ]:
def get_similarity(word, X, X_vocab, vocab):       
    # Create a dict to find word index
    vocab_d = {}
    for i, w in enumerate(X_vocab):
        vocab_d[w] = i
        
    # Get the similarity for the word
    y = X[vocab_d[word]].reshape(1, -1)
    res = cosine_similarity(X, y)
    df = pd.DataFrame(columns=['word', 'sim'])
    df['word'], df['sim']= vocab[1:], res
    return(df)

In [ ]:
def get_3d_viz(X, X_vocab):
    
    pca = PCA(n_components=3)
    pca_embed = pca.fit_transform(X)
    
    df = pd.DataFrame(columns=['x', 'y', 'z', 'word'])
    df['x'], df['y'], df['z'], df['word'] = pca_embed[:,0], pca_embed[:,1], pca_embed[:,2], X_vocab
    fig = px.scatter_3d(df, x='x', y='y', z='z', color='word')
    return(fig)

In [ ]:
class TrackLossAndSimilarity(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        neptune.log_metric('loss', logs["loss"])
        neptune.log_metric('accuracy', logs["accuracy"])
        # Log the similarity to an example word at regular intervals
        if epoch%100 == 0:
            vocab = vectorize_layer.get_vocabulary()
            X, X_vocab = get_data(word2vec.get_layer('w2v_embedding').get_weights()[0], 
                                  vocab)
            check_word = 'bbb'
            sim_df = get_similarity(check_word, X, X_vocab, vocab)
            sim_fig = get_3d_viz(X, X_vocab)
            log_chart(f'3d-plot-epoch{epoch}', sim_fig)
            log_table(f'similarity-{check_word}-epoch{epoch}', sim_df.sort_values('sim', ascending=False))

Train the model with `dataset` prepared above for some number of epochs.

In [ ]:
word2vec.fit(dataset, epochs=1000, callbacks=[TrackLossAndSimilarity()])